In [1]:
!pip install plotly "jupyterlab>=3" "ipywidgets>=7.6"


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import plotly
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
import json
import pandas as pd
from IPython.display import display, HTML

In [5]:
with open('Userdata.json', 'r', encoding='utf8') as userdata_file:
    user_data = json.load(userdata_file)
    
with open('Inferences.json', 'r', encoding='utf8') as inference_file:
    inference_data = json.load(inference_file)
    
with open('StreamingHistory.json', 'r', encoding='utf8') as streaminghistory_file:
    streaminghistory_data = json.load(streaminghistory_file)

In [6]:
# user data
def get_user_data():
    grid_items = []
    for key, value in user_data.items():
        key_widget = widgets.Label(value=key.capitalize())
        value_widget = widgets.Label(value=str(value))
        grid_items.extend([key_widget, value_widget])
    grid = widgets.GridBox(grid_items, layout=widgets.Layout(grid_template_columns="repeat(2, 250px)"))
    return grid

# inference data
def get_inference_data():
    data = {"1P": [], "3P":[]}
    for value in inference_data['inferences']:
        data[value[:2]].append(value[3:])
    
    labels = {
        "1P": widgets.Box([widgets.Label(v) for v in data["1P"]]),
        "3P": widgets.VBox([widgets.Label(v) for v in data["3P"]])
    }

    accordion = widgets.Accordion(children=list(labels.values()))
    accordion.set_title(0, "First Party")
    accordion.set_title(1, "Third Party")
    
    return accordion


# streaming data
def get_streaming_data():
    df = pd.DataFrame(streaminghistory_data)
    
    # PlayTime
    artist_playtime = df.groupby('artistName')['msPlayed'].sum().reset_index()
    artist_playtime["msPlayed"] = artist_playtime["msPlayed"] / 60000
    trace = go.Bar(x=artist_playtime["artistName"], y=artist_playtime["msPlayed"])
    layout = go.Layout(
        title="Total Playtime by Artist",
        xaxis=dict(title="Artist Name"),
        yaxis=dict(title="Total Playtime (minute)")
    )
    ap_fig = go.FigureWidget(data=[trace], layout=layout)

    # Artist wise songs
    artist_trackName_df = df.groupby('artistName')['trackName'].unique().reset_index()
    table_trace = go.Table(
        header=dict(values=["Artist Name", "Track Name"]),
        cells=dict(values=[artist_trackName_df.artistName, artist_trackName_df.trackName.str.join('<br>')])
    )
    layout = go.Layout(title="Streaming Data Table")
    at_fig = go.FigureWidget(data=[table_trace], layout=layout)
    
    selection = widgets.Dropdown(options=['All', *sorted(df["artistName"].unique())],
                                 value='All',
                                 description='Artist',
                                 disabled=False)

    def selection_change(change):
        with at_fig.batch_update():
            if change.new == "All":
                tempr = artist_trackName_df
            else:
                tempr = artist_trackName_df[artist_trackName_df["artistName"] == change.new]
            at_fig.data[0].cells = dict(values=[tempr.artistName, tempr.trackName.str.join('<br>')])
        

    selection.observe(selection_change, names="value")
    
    
    # add tabs
    tab = widgets.Tab()
    tab.children = [ap_fig, widgets.VBox([selection, at_fig])]
    tab.titles = ["PlayTime Chart", "Artists & Songs"]
    
    return tab


def show_data():
    tab = widgets.Tab()
    tab.children = [get_user_data(), get_inference_data(), get_streaming_data()]
    tab.titles = ["User Data", "Inference Data", "Streaming History"]
    display(tab)
    
show_data()